In [3]:
# !pip install -r requirements.txt
#맨아래에 tree 를 적어두었으니, 어떻게 데이터 파일이 넣어져있는지 궁금하면, 해당 것 다운로드해서 봐도됨.
#train.py에 에러날시에 다음과 같은 문장을 삽입할 것.
#import numpy as np ## Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL 

In [57]:
!nvidia-smi

Thu Nov 26 14:08:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
|  0%   28C    P8     2W / 250W |   1550MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:D9:00.0 Off |                  N/A |
| 27%   28C    P8     3W / 250W |     10MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [5]:
!pwd

/data2/home/jihye/Desktop/VizWiz-VQA-PyTorch


In [36]:
# !mkdir data_vizwiz/Images
## data download

In [54]:
## 중간에 외부 module이 인식 못하는 error 발생시에는 해당 것을 해결하기 위해서 그냥 직접 붙여넣어라.
# !pip install pandas==0.23.0

# 자신의 쿠다에 맞게 알아서 할 것!
# !pip install torch==1.6.0
# !pip install torchvision==0.7.0

!python ./preprocessing/image_features_extraction.py

Found 39704 images in ./data_vizwiz/Images
./preprocessing/image_features_extraction.py:142: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  dummy_input = Variable(torch.ones(1, 3, config['img_size'], config['img_size']), volatile=True).cuda()
Extracting features ...
  0%|                                                  | 0/9926 [00:00<?, ?it/s]./preprocessing/image_features_extraction.py:172: UserWarning: This overload of cuda is deprecated:
	cuda(torch.device device, bool async, *, torch.memory_format memory_format)
Consider using one of the following signatures instead:
	cuda(torch.device device, bool non_blocking, *, torch.memory_format memory_format) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  inputs_img = Variable(inputs['visual'].cuda(async=True), volatile=True)
./preprocessing/image_features_extraction.py:172: UserWarning: volatile was removed and now ha

In [26]:
!python ./preprocessing/create_vocabs.py

{'annotations': {'dir': './data_vizwiz/Annotations',
                 'max_length': 26,
                 'min_count_word': 0,
                 'path_vocabs': './prepro_data/vocabs.json',
                 'top_ans': 3000},
 'images': {'arch': 'ResNet152',
            'dir': './data_vizwiz/Images',
            'img_size': 448,
            'mode': 'att',
            'path_features': './prepro_data/resnet14x14.h5',
            'preprocess_batch_size': 4,
            'preprocess_data_workers': 4},
 'logs': {'dir_logs': 'logs/vizwiz/'},
 'model': {'attention': {'dropout': 0.5, 'glimpses': 2, 'mid_features': 512},
           'classifier': {'dropout': 0.5},
           'pooling': {'dim_h': 1024,
                       'dim_q': 1024,
                       'dim_v': 2048,
                       'dropout_q': 0.5,
                       'dropout_v': 0.5},
           'pretrained_model': None,
           'seq2vec': {'dropout': 0.25, 'emb_size': 300}},
 'prediction': {'model_path': './logs/vizwiz/2018

In [56]:
!python train.py

Model logs will be saved in logs/vizwiz/2020-11-26_00:01:34
train.py:215: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  if len(self.answers[i].nonzero()) > 0:
number of total epochs : 300
train E000:   0% 0/156 [00:00<?, ?it/s]/data2/home/jihye/anaconda3/envs/test/lib/python3.6/site-packages/torch/nn/modules/rnn.py:580: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/aten/src/ATen/native/cudnn/RNN.cpp:1234.)
  self.num_layers, self.dropout, self.training, self.bidirectional)
train E000: 100% 156/156 [00:23<00:00,  6.6

In [61]:
#!mkdir predictions
!python predict.py

Loading samples to predict from ./data_vizwiz/Annotations/test.json
Loading Model from ./logs/vizwiz/2020-11-26_00:01:34/best_accuracy_log.pth
  0%|                                                    | 0/63 [00:00<?, ?it/s]Evaluating...

predict.py:264: UserWarning: This overload of cuda is deprecated:
	cuda(torch.device device, bool async, *, torch.memory_format memory_format)
Consider using one of the following signatures instead:
	cuda(torch.device device, bool non_blocking, *, torch.memory_format memory_format) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  v = Variable(v.cuda(async=True))
/data2/home/jihye/anaconda3/envs/test/lib/python3.6/site-packages/torch/nn/modules/rnn.py:580: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (

In [68]:
# 별도의 test가 불가능하다. 왜냐하면, test sets의 answer이 없기 때문에! (따라서, vizwiz 사이트에서 평가필요.)
#!python eval_test.py

Model logs will be saved in logs/vizwiz/2020-11-26_14:37:42
Loading Model from ./logs/vizwiz/2020-11-26_00:01:34/best_accuracy_log.pth
test E000:   0% 0/63 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "eval_test.py", line 357, in <module>
    main()
  File "eval_test.py", line 353, in main
    eval_results = evaluate(model, test_loader, tracker, epoch=0, split='test')
  File "eval_test.py", line 268, in evaluate
    a = item['answer']
KeyError: 'answer'


In [72]:
!tree

.
├── config
│   └── default.yaml
├── datasets
│   ├── features.py
│   ├── images.py
│   ├── __pycache__
│   │   └── images.cpython-36.pyc
│   └── vqa_dataset.py
├── data_vizwiz
│   ├── Annotations
│   │   ├── test.json
│   │   ├── train.json
│   │   └── val.json
│   └── Images
│       ├── VizWiz_test_00000000.jpg
│       ├── VizWiz_test_00000001.jpg
│       ├── VizWiz_test_00000002.jpg
│       ├── VizWiz_test_00000003.jpg
│       ├── VizWiz_test_00000004.jpg
│       ├── VizWiz_test_00000005.jpg
│       ├── VizWiz_test_00000006.jpg
│       ├── VizWiz_test_00000007.jpg
│       ├── VizWiz_test_00000008.jpg
│       ├── VizWiz_test_00000009.jpg
│       ├── VizWiz_test_00000010.jpg
│       ├── VizWiz_test_00000011.jpg
│       ├── VizWiz_test_00000012.jpg
│       ├── VizWiz_test_00000013.jpg
│       ├── VizWiz_test_00000014.jpg
│       ├── VizWiz_test_00000015.jpg
│       ├── VizWiz_test_00000016.jpg
│       ├── VizWiz_test_00000017.jpg
│       ├── VizWiz_test_00000018.jpg
│       ├── VizWiz